## Network A: Linking the road network and land use data

**TU Delft**<br>
**Author:** Ruth Nelson <br>

The main point of this notebook is to download the street network, clean it, link it to the land use data 
using Snkit and creating a graph from it.

We will be using the Cape Town data set as an example.

1. Import libraries
2. Download Cape Town street network from OSMNX
3. Cleaning the network by consolidating intersections
4. Transform OSMNX graph to geopanda
5. Import and clean landuse data
6. Concattenating the land use data to the street vertices
7. Snkit to link land use vertices
8. Saving the files to shp

In [ ]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import snkit
import osmnx as ox
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.notebook_repr_html', True)

### 2. Download street network from OSMNX

In [ ]:
# download data from osm

In [ ]:
G = ox.graph_from_bbox(-33.78336, -34.15007,18.34072, 18.92836, network_type='drive') #used coordinates to delineate area
G_projected = ox.project_graph(G)
ox.plot_graph(G_projected)

### 3. Cleaning the network by consolidating intersections

In [ ]:
# get a GeoSeries of consolidated intersections
ints = ox.consolidate_intersections(G_projected, rebuild_graph=False, tolerance=15, dead_ends=False)
len(ints)

In [ ]:
# compare to number of vertices in original graph
len(G)

In [ ]:
# consolidate intersections and rebuild graph topology
# this reconnects edge geometries to the new consolidated vertices
G2 = ox.consolidate_intersections(G_projected, rebuild_graph=True, tolerance=15, dead_ends=False)
len(G2)

### 4. Transform OSMNX graph to geopanda

In [ ]:
# you can convert your graph to vertex and edge GeoPandas GeoDataFrames
gdf_vertices, gdf_edges = ox.graph_to_gdfs(G2)
gdf_vertices.head()

In [ ]:
gdf_edges.head()

### 5. Import and clean land use data:

- will use official land use data (over points of interest to see what is important)
- will link using sknit

In [ ]:
path_to_land = ""

In [ ]:
os.chdir(path_to_land)
for file in glob.glob("*"):
    print(file)

In [ ]:
#reading in the landuse data, which are centroids of all non-residential land use
landuse = gpd.read_file('land_use.shp') 

### 6. Concattenating the land use data to the street vertices

In [ ]:
#Geometries to same crs
landuse2 = landuse.to_crs(3857)

In [ ]:
landuse2['x'] = landuse2.geometry.x # x coord of column
landuse2['y'] = landuse2.geometry.y #y coord of column

In [ ]:
landuse2.head()

In [ ]:
#making sure its the same crs as the landuse
gdf_vertices_2 = gdf_vertices.to_crs(3857)

In [ ]:
gdf_vertices_2.head()

In [ ]:
gdf_vertices_2['x'] = gdf_vertices_2.geometry.x # x coord of column
gdf_vertices_2['y'] = gdf_vertices_2.geometry.y #y coord of column

In [ ]:
#making sure its the same crs as the landuse
gdf_edges_2 = gdf_edges.to_crs(3857)

In [ ]:
gdf_edges_2.head()

Concatenate land use vertices and street vertices

In [ ]:
vertices_network = pd.concat([landuse2,gdf_vertices_2])

In [ ]:
vertices_network = vertices_network.reset_index().drop('index', axis=1)

### 7. Snkit to link land use vertices

I will now link the land use vertices to the street network utilising snkit

In [ ]:
#Create basenetwork for snkit
base_network = snkit.Network(vertices_network, gdf_edges_2) #this adds the additional edges

In [ ]:
# # Plot the edges and vertices
ax = plt.axes()
base_network.edges.plot(ax=ax, linewidth = 1, alpha = 0.6)
base_network.nodes.plot(ax=ax, facecolor = 'red', markersize = 3)

In [ ]:
base_network.edges = base_network.edges.reset_index()

In [ ]:
#linking the landuse to the network

In [ ]:
linked = snkit.network.link_nodes_to_nearest_edge(base_network, condition=None)
#ignore the warnings

In [ ]:
#plot
ax = plt.axes()
linked.edges.plot(ax=ax, linewidth = 1, alpha = 0.5)
linked.nodes.plot(ax=ax, facecolor = 'red', markersize = 3, alpha = 1)

In [ ]:
#Working with this new network

In [ ]:
# create IDs for the new vertices and edges 
with_id = snkit.network.add_topology(snkit.network.add_ids(linked))

In [ ]:
## Adjusting attributes 

In [ ]:
with_id.nodes

In [ ]:
with_id.nodes['land use'] = with_id.nodes['land use'].fillna('None')

In [ ]:
#identifying a vertex as belonging to a street

In [ ]:
with_id.nodes.osmid_original = with_id.nodes.osmid_original.fillna(0) #NANs become O

In [ ]:
with_id.nodes.loc[with_id.nodes['osmid_original'] != 0, 'osmid_original'] = 'street' #identifies all street vertices

In [ ]:
with_id.nodes.loc[with_id.nodes['osmid_original'] == 0, 'osmid_original'] = 'not_street' #identifies all non-street vertices

In [ ]:
with_id.nodes = with_id.nodes.rename(columns = {'osmid_original':'vertex_type'}) #change column name to what it is

In [ ]:
#adding a geometry to each vertex

In [ ]:
with_id.nodes['x'] = with_id.nodes['geometry'].x # x coord of column
with_id.nodes['y'] = with_id.nodes['geometry'].y #y coord of column

In [ ]:
relevant_columns = ['id','land use','vertex_type', 'x', 'y', 'geometry']

#reduce vertices to relevant columns only
vertices = with_id.nodes[relevant_columns]

In [ ]:
#classifies all land use connectors (the created edges)
with_id.edges['length'] = with_id.edges['length'].fillna(0)
with_id.edges.loc[with_id.edges['length'] == 0, 'highway'] = 'land_use_connector'

In [ ]:
#rename highwy to edge_type
with_id.edges = with_id.edges.rename(columns = {'highway':'edge_type'})
#make sure it is in string format
with_id.edges['edge_type']=with_id.edges['edge_type'].astype(str)

In [ ]:
with_id.edges

In [ ]:
#reduce to relevant columns
relevant_columns = ['length','edge_type', 'id', 'from_id', 'to_id', 'geometry']
edges = with_id.edges[relevant_columns]

In [ ]:
#have a column identifying if an edge is a street or not
edges['vertex_type'] = 'street' #create a column identifying if the edge is a street or not with default value as street

In [ ]:
edges

In [ ]:
edges.loc[edges['length'] == 0, 'vertex_type'] = 'not_street' #identifies all not street edges

drop the length column as it is no longer accurate due to each street 
edge being divided by the connecting land use edges and vertices

In [ ]:
edges = edges.drop('length', axis=1)

In [ ]:
vertices

### 8. Saving the files to shp

In [ ]:
path_to_save = " "

In [ ]:
os.chdir(path_to_save)
for file in glob.glob("*"):
    print(file)

In [ ]:
edges.to_file('edges_a.shp')

In [ ]:
vertices.to_file('vertices_a.shp')